In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from multiprocessing import Pool
from collections import ChainMap
from joblib import Parallel, delayed
from scipy.special import logit, expit

import warnings
warnings.filterwarnings("ignore")
from sir_model import SusInfRec
from mpl_toolkits.mplot3d import Axes3D

In [2]:
airport_connections = pd.read_csv("../data/connections_preprocessed.csv", index_col=0)
airport_connections.head()

,flights_per_month,country,airline,airline_id,source_airport,source_airport_id,dest_airport,dest_airport_id,latitude_source,longitude_source,latitude_dest,longitude_dest
0,88,United States,DL,2009,ABE,4355,DTW,3645,40.652100,-75.440804,42.212399,-83.353401
1,90,United States,DL,2009,ALB,3864,DTW,3645,42.748299,-73.801697,42.212399,-83.353401
2,53,United States,DL,2009,APN,5720,DTW,3645,45.078098,-83.560303,42.212399,-83.353401
3,337,United States,AF,137,ATL,3682,DTW,3645,33.636700,-84.428101,42.212399,-83.353401
4,337,United States,AM,321,ATL,3682,DTW,3645,33.636700,-84.428101,42.212399,-83.353401


In [3]:
airport_data = pd.read_csv("../data/airport_data_preprocessed.csv", index_col=0)
airport_data.head()

,ID,Name,City,Country,IATA,Lat,Long,population,source_airport,flights_per_month,flights_coefficient
0,3411,Barter Island LRRS Airport,Barter Island,United States,BTI,70.134003,-143.582001,9121.0,BTI,27.0,337.810107
1,3413,Cape Lisburne LRRS Airport,Cape Lisburne,United States,LUR,68.875099,-166.110001,1352.0,LUR,4.0,337.810107
2,3414,Point Lay LRRS Airport,Point Lay,United States,PIZ,69.732903,-163.005005,9121.0,PIZ,27.0,337.810107
3,3415,Hilo International Airport,Hilo,United States,ITO,19.721399,-155.048004,43263.0,ITO,634.0,337.810107
5,3417,Bettles Airport,Bettles,United States,BTT,66.913902,-151.529007,10473.0,BTT,31.0,337.810107


In [4]:
"""
Below functions are defined here.
a: Probability of Infected Plane
b: Probability of Infected City
"""

"""
probability_of_infected_plane:
    I is the number of people infected in the city
    N is the total population in the city
"""
def probability_of_infected_plane(I, N):
    return I/N

"""
probability_of_infected_city:
    Find the probability of a city to get infected from an incoming plane

    P(new city infected) = 1 - P(all incoming planes are healthy)
        = 1 - [P(all planes from city A are healthy) * P(all planes from city B are healthy) * ...]
        = 1 - [(1 - I_A/N_A) ^ f_A * (1 - I_B/N_B) ^ f_B * ...]
"""
def probability_of_infected_city(infectious_sources, populations_sources, daily_flights_sources):
    prob_all_planes_healthy = 1
    for I, N, f in zip(infectious_sources, populations_sources, daily_flights_sources):    
        prob_all_planes_healthy *= (1-probability_of_infected_plane(I, N)) ** f
        
    return 1 - prob_all_planes_healthy

## Compute Reproduction Number

We are dynamically modelling the reproduction number. The reproduction number depends upon the intervention of government on the activity of stopping planes and starting quarintine measures and eventually enforcing a lockdown. We choose a range of max R to min R. Assuming there is no vaccination of this disease, the min R will be 1.

* interventionStart: number of days after which reproduction number starts going down
* interventionPeriod: number of days it takes from the intervention start to drive R value to its minimal values
* simulationDays: number of days in the simulation run
* maxR: maximal reproduction number value during a period of simulation days
* minR: minimal possible reproduction number value during a period of simulation days

##### Algorithm
* Create an array of maxR repeated till intervention start period


In [5]:
def compute_reproduction_number(
    interventionStart, 
    interventionPeriod,
    simulationDays,
    maxR, 
    minR
):
    
    # Part A
    # Repeat elements of array reproduction_n ith values max_R till intervention_start
    reproductionNumber = np.repeat(maxR, interventionStart)
    
    # Part B
    # expit(x) = 1/(1+exp(-x)). It is the inverse of the logit function.
    # reproductionIntervention is the period in which the R value starts decreasing from maxR to minR
    reproductionIntervention = expit(np.linspace(-5, 3, num=interventionPeriod))[::-1]
    reproductionIntervention = reproductionIntervention * (maxR - minR) + minR
    
    # Part C
    # Once we reach the min R, continue minR till end of the simulation days, assuiming lockdown is enforced and people
    # are quarantining till the end of tthe simulation days
    
    # Concatenating the range of Part A, Part B and Part C to get the entire R for a duration of simulation days
    reproductionNumber = np.concatenate(
        (
            reproductionNumber, 
            reproductionIntervention,
            np.repeat(minR, simulationDays)
        )
    )
    return reproductionNumber

In [6]:
# taking unique source-destination pairs
airport_connections = airport_connections.groupby(["source_airport", 'dest_airport'], as_index=False).agg({
    "flights_per_month":np.nansum,
    "latitude_source":min,
    "longitude_source":min,
    "latitude_dest":min,
    "longitude_dest":min
})

airport_connections_graph = nx.from_pandas_edgelist(
    airport_connections, 
    source = 'source_airport', 
    target = 'dest_airport',
    create_using = nx.DiGraph()
)

In [7]:
"""
city_neighbours:
    Find neighbouring cities for a given city by searching in the DiGraph
"""
def city_neighbours(city_name, airport_connections_graph=airport_connections_graph):
    city_airports = CITY_TO_AIRPORT_CODE[city_name]
    neighbours = []
    for city in city_airports:
        """
        Digraph.neighbors returns all the neighbors of a given node
        """
        neighbours.extend(list(airport_connections_graph.neighbors(city)))
    return neighbours

In [8]:
"""
Dict of Airport Code and their corresponding cities

Eg: {
        'BOS': 'Boston',
        'PWM': 'Portland',
        'PDX': 'Portland'
    }
"""
AIRPORT_CODE_TO_CITY = airport_data[['City', 'IATA']].set_index("IATA").to_dict()['City']


"""
Dict of City and their corresponding airport codes (Few Cities have more than 1 airports)

Eg: {
        'Boston': ['BOS'],
        'Portland': ['PDX', 'PWM'] 
    }
"""
CITY_TO_AIRPORT_CODE = airport_data[['City', 'IATA']].groupby("City")['IATA'].unique().to_dict()
CITY_TO_AIRPORT_CODE = {k:list(v) for k, v in CITY_TO_AIRPORT_CODE.items()}

"""
Dict of Cities and their resp populations
Eg: {
        'Boston': 667137.0,
        'Portland': 632309.0
    }
"""
CITY_POPULATION = airport_data[['City', 'population']].set_index("City").to_dict()['population']

"""
Dict of Cities and their resp neighbors (Direct Flight connections)
Eg: {
        'Boston': ['ATL', 'BWI', 'CLT', 'DCA', 'DEN','DFW', 'DTW', 'EWR', 'FLL', 'JFK', 'LAX', 'LEB', 'MCO', 'MIA', 'ORD',
                    'PHL', 'RDU', 'RSW', 'SEA', 'SFO', 'TPA'],
        'Portland': ['ATL', 'BOI', 'DEN', 'DFW', 'GEG', 'LAS', 'LAX', 'MSP', 'ORD', 'PDT', 'PHX', 'PSP', 'RDM', 'RNO', 
                        'SAN', 'SEA', 'SFO', 'SJC', 'SLC', 'SMF', 'SNA', 'BWI', 'JFK']
    }
    
"""

CITY_NEIGHBOURS = {}
for city in airport_data.City.unique():
    try:
        CITY_NEIGHBOURS[city] = city_neighbours(city)
    except:
        continue

"""
Number of flights between source & destination every month

Eg: {
         ('BOS', 'ATL'): 58.8,
         ('BOS', 'BWI'): 14.8,
         ('BOS', 'CLT'): 21.400000000000002,
         ('BOS', 'DCA'): 17.1,
         ('BOS', 'DEN'): 13.6,
         ('BOS', 'DFW'): 20.4,
         ('BOS', 'DTW'): 8.8
    }
"""
NUMBER_OF_FLIGHTS = dict(zip(tuple(
    zip(
        airport_connections.source_airport, 
        airport_connections.dest_airport
    )),
    airport_connections.flights_per_month
))

In [9]:
"""
This function is used to pull out the neighboring cities from the City_Neighbors data frame. 
"""
def find_neighbours_of_a_city(city_name):
    return CITY_NEIGHBOURS[city_name]

"""
This provides the list of healthy airports by removing the set of infected airports from overall list.
"""
def find_all_current_healthy_airports(airports):
    airports = list(set(airports) - set(INFECTED_AIRPORTS))
    return airports

"""
This provides the list of infected airports by creating an intersection between the overall list of airports with the set of Infected airports.
"""
def find_all_current_infected_airports(airports):
    airports = list(set(airports).intersection(set(INFECTED_AIRPORTS)))
    return airports


def airports_to_cities(airports):
    return list(set([AIRPORT_CODE_TO_CITY[code] for code in airports]))

"""
This function pulls out the number of flights operating between a source and destination.
"""
def find_total_number_of_flights(source, destination):
    if not isinstance(source, list):
        source = [source]
    if not isinstance(destination, list):
        destination = [destination]
    flights = 0
    for source in source:
        for dest in destination:
            flights+=NUMBER_OF_FLIGHTS[(source, dest)]
    return flights


"""
It is used to extract the information that provides us with the infections on a day according to the infections happening in the cities on a particular day.
"""
def get_infected_number(city_name, simulation_day):
    infection_day = INFECTED_CITIES[city_name]['day']
#     infected_number = INFECTED_CITIES[city_name]['infected'][simulation_day-infection_day]
#     print("Day - ",simulation_day," :City - ",city_name," :Infected_number - ",infected_number)
    return INFECTED_CITIES[city_name]['infected'][simulation_day-infection_day]

"""
This function performs the operation of calculating the probability of the spread of infection based on important factors like current susceptible city, the list of infected population, the infected number, the infected city, etc.
"""
def calculate_infection_prob(current_susceptible_city, DAY):
    
    current_susceptible_airports = CITY_TO_AIRPORT_CODE[current_susceptible_city]
    current_infected_neighbours = find_all_current_infected_airports(find_neighbours_of_a_city(current_susceptible_city))

    flights = []
    infected_populations = []
    total_populations = []

    for infected_neighbour in current_infected_neighbours:
        infected_city_name = AIRPORT_CODE_TO_CITY[infected_neighbour]
        flights.append(find_total_number_of_flights(infected_neighbour, current_susceptible_airports))
        infected_populations.append(get_infected_number(infected_city_name, DAY))
        total_populations.append(CITY_POPULATION[infected_city_name])

    infection_probability = probability_of_infected_city(infected_populations, total_populations, flights)
    return infection_probability

"""
This is used to create a simulation based on the S.I.R model that we have implemented. 
This simulation helps in understanding the spread of the disease on a day-to-day basis based on some key factors like day, 
infected, susceptible, recovered, etc. This SIR model is processed by providing the city population 
and an assumed figure of the first infected number (which we have considered as 100). 
"""
def run_neighbour_simulation(current_susceptible_city, current_infection_source_city, DAY):
    
    infection_probability = calculate_infection_prob(current_susceptible_city, DAY)

    if np.random.random() < infection_probability:
        S, I, R = run_sir(
            city_population=CITY_POPULATION[current_susceptible_city],
            first_infected_number=100
        )
#         print("current_susceptible_city : ", current_susceptible_city, 'day', DAY, 'from', current_infection_source_city)
        return {current_susceptible_city:{
            'day':DAY, 
            'infected':I,
            'susceptible':S,
            'recovered':R,
            'from': current_infection_source_city
        }}
        
"""
In this function we are able to pull out the information about the infectious cities based on 
the commutes that are happening through air travel. 
The information needed to perform this simulation is the neighboring airports, the susceptible airports, 
the susceptible cities, the current infection city source etc. 
"""     
def run_infectious_city_simulation(current_infection_source_city, DAY):
    neighbour_airports = find_neighbours_of_a_city(current_infection_source_city)
    susceptible_airports = find_all_current_healthy_airports(neighbour_airports)
    susceptible_cities = airports_to_cities(susceptible_airports)
#     print(susceptible_cities)

    results = []
    for current_susceptible_city in tqdm_notebook(
        susceptible_cities, 
        leave=False, 
        desc='susceptible', 
        disable=True
    ):
        
        try:
            results.append(run_neighbour_simulation(current_susceptible_city, current_infection_source_city, DAY))
        except:
            continue
            
    results = [res for res in results if res]
    return results

In [10]:
NUMBER_OF_SIMULATIONS = 2
SIMULATION_DAYS = 100
VERBOSE = True

maxR = 5
minR = 1

interventionStart = 10
interventionPeriod = 15

"""
These assumed values are now fed into our model through various functions starting with 
equating the reproduction number of the disease through calculate_reproduction_number() function 
that takes 5 arguments namely max_R, min_R, simulation_days, intervention_start, intervention_period. 

The next function to be executed is the main function of our S.I.R model. 
This function will return the S, I & R values of the disease which will be used in the next function 
to perform the appropriate simulation. This function takes in values of city_population, first_infected_number, 
reproduction_number, gamma, simulation_days. 

The final task of this section is to now perform a simulation based on the computations that we have done so far. 
In order to execute that it will need the following information infected cities, infected airports, new infections, 
the value of S, I, R, susceptible cities, infected cities, current infection source etc. 
As we have already computed the above information, we have used TQDM_Notebook to perform the simulation task. 
TQDM notebook is a Jupyter Notebook progress bar decorator for iterators. 
"""

GAMMA = 0.050
REPRODUCTION_NUMBER = compute_reproduction_number(
    interventionStart, 
    interventionPeriod,
    SIMULATION_DAYS,
    maxR, minR  )

def run_sir(
        city_population, first_infected_number, 
        reproduction_number=REPRODUCTION_NUMBER, 
        gamma=GAMMA, days=SIMULATION_DAYS
    ):
    sir_model = SusInfRec(
        city_population, I0=first_infected_number, b=reproduction_number/15, k=gamma, days=days
    )
    S, I, R = sir_model.run()
    return S, I, R

In [11]:
INFECTED_CITIES = {}
INFECTED_AIRPORTS = []
NEW_INFECTED = {}

for simulation_run in tqdm_notebook(range(NUMBER_OF_SIMULATIONS), leave=False):
    # Always start at Boston on day 0
    S, I, R = run_sir(CITY_POPULATION['Boston'], 10)
    INFECTED_CITIES = {'Boston':{'day':0, 'infected':I, 'susceptible':S, 'recovered':R, 'from':'Boston'}}
    INFECTED_AIRPORTS = ['BOS']
    
    for DAY in tqdm_notebook(range(0, SIMULATION_DAYS), desc='Day', leave=False):
        CHECKED_SUSCEPTIBLE_CITIES = []

        for current_infection_source_city in tqdm_notebook(
            INFECTED_CITIES.keys(),
            leave=True, 
            desc='infection sources',
            disable=not VERBOSE
            ):

            results = run_infectious_city_simulation(current_infection_source_city, DAY)
            NEW_INFECTED.update(dict(ChainMap(*results)))

        INFECTED_CITIES.update(NEW_INFECTED)
        NEW_INFECTED = {}
        INFECTED_AIRPORTS = sum([CITY_TO_AIRPORT_CODE[city] for city in INFECTED_CITIES.keys()], [])
        
#         print("Day -- ", DAY)
    with open(f"../data/INFECTED_CITIES_{simulation_run}", 'wb') as f:
        pickle.dump(INFECTED_CITIES, f)